# 03_model_development

## Carbon Sequestration Prediction Modeling

**Objectives:**
- Train carbon sequestration prediction models
- Compare multiple algorithms (Random Forest, XGBoost, Neural Networks)
- Hyperparameter tuning with cross-validation
- Comprehensive model evaluation
- Feature importance analysis

**Target Variable:** Carbon Stock (kg/m²)

## 1. Import Dependencies and Setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Machine Learning
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder, PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.inspection import permutation_importance

# Advanced Models
import xgboost as xgb
import lightgbm as lgb
from sklearn.ensemble import StackingRegressor

# Visualization
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import joblib

# Setup
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
%matplotlib inline
np.random.seed(42)

print("✅ All dependencies imported successfully")

## 2. Data Loading and Preparation

In [ ]:
# Load processed data from EDA
try:
    biomass_df = pd.read_csv('outputs/biomass_data_with_insights.csv')
    json_df = pd.read_csv('outputs/json_data_with_insights.csv')
    print("✅ Successfully loaded processed data from EDA")
except FileNotFoundError:
    # Fallback to original data
    try:
        biomass_df = pd.read_csv('data/processed_biomass_csv.csv')
        json_df = pd.read_csv('data/processed_biomass_json.csv')
        print("✅ Loaded processed data from data directory")
    except FileNotFoundError:
        print("❌ No processed data found. Please run previous notebooks first.")
        raise

# Combine datasets for larger training set
def prepare_combined_dataset(csv_df, json_df):
    """Combine and prepare datasets for modeling."""
    
    # Standardize column names
    if 'biomass_value' in csv_df.columns:
        csv_df = csv_df.rename(columns={'biomass_value': 'biomass'})
    
    # Select common columns
    common_cols = ['biomass', 'carbon_stock', 'latitude', 'longitude']
    
    # Add vegetation type if available
    if 'vegetation_type' in csv_df.columns:
        common_cols.append('vegetation_type')
    
    if 'quality_flag' in json_df.columns:
        common_cols.append('quality_flag')
    
    # Combine datasets
    combined_df = pd.concat([
        csv_df[common_cols],
        json_df[common_cols]
    ], ignore_index=True)
    
    return combined_df

# Prepare combined dataset
modeling_df = prepare_combined_dataset(biomass_df, json_df)
print(f"📊 Combined dataset shape: {modeling_df.shape}")
print(f"📈 Target variable: carbon_stock")
print(f"🎯 Target distribution:")
print(modeling_df['carbon_stock'].describe())

# Display first few rows
print("\nFirst 5 rows of combined data:")
display(modeling_df.head())

## 3. Feature Engineering

In [ ]:
class CarbonFeatureEngineer:
    """Feature engineering for carbon sequestration prediction."""
    
    def __init__(self):
        self.label_encoders = {}
        self.scaler = StandardScaler()
    
    def create_spatial_features(self, df):
        """Create spatial and geographic features."""
        df_eng = df.copy()
        
        # Spatial clustering features
        df_eng['lat_bin'] = pd.cut(df_eng['latitude'], bins=5, labels=False)
        df_eng['lon_bin'] = pd.cut(df_eng['longitude'], bins=5, labels=False)
        
        # Distance from center
        center_lat = df_eng['latitude'].mean()
        center_lon = df_eng['longitude'].mean()
        df_eng['distance_from_center'] = np.sqrt(
            (df_eng['latitude'] - center_lat)**2 + 
            (df_eng['longitude'] - center_lon)**2
        )
        
        # Spatial interaction terms
        df_eng['lat_lon_interaction'] = df_eng['latitude'] * df_eng['longitude']
        df_eng['spatial_cluster'] = (df_eng['lat_bin'] * 10 + df_eng['lon_bin']).astype(int)
        
        return df_eng
    
    def create_biomass_features(self, df):
        """Create biomass-derived features."""
        df_eng = df.copy()
        
        # Biomass transformations
        df_eng['log_biomass'] = np.log1p(df_eng['biomass'])
        df_eng['sqrt_biomass'] = np.sqrt(df_eng['biomass'])
        df_eng['biomass_squared'] = df_eng['biomass'] ** 2
        
        # Biomass categories
        df_eng['biomass_category'] = pd.cut(
            df_eng['biomass'], 
            bins=[0, 1, 2, 3, 5, np.inf], 
            labels=['Very_Low', 'Low', 'Medium', 'High', 'Very_High']
        )
        
        return df_eng
    
    def encode_categorical_features(self, df):
        """Encode categorical variables."""
        df_eng = df.copy()
        
        categorical_columns = ['biomass_category']
        if 'vegetation_type' in df_eng.columns:
            categorical_columns.append('vegetation_type')
        if 'quality_flag' in df_eng.columns:
            categorical_columns.append('quality_flag')
        
        for col in categorical_columns:
            if col in df_eng.columns:
                le = LabelEncoder()
                df_eng[col + '_encoded'] = le.fit_transform(df_eng[col].astype(str))
                self.label_encoders[col] = le
        
        return df_eng
    
    def create_interaction_features(self, df):
        """Create interaction features between variables."""
        df_eng = df.copy()
        
        # Biomass-spatial interactions
        df_eng['biomass_latitude'] = df_eng['biomass'] * df_eng['latitude']
        df_eng['biomass_longitude'] = df_eng['biomass'] * df_eng['longitude']
        
        # Polynomial features for key variables
        poly = PolynomialFeatures(degree=2, include_bias=False, interaction_only=True)
        interaction_features = poly.fit_transform(df_eng[['biomass', 'latitude', 'longitude']])
        interaction_df = pd.DataFrame(
            interaction_features, 
            columns=poly.get_feature_names_out(['biomass', 'latitude', 'longitude'])
        )
        
        df_eng = pd.concat([df_eng, interaction_df], axis=1)
        
        return df_eng
    
    def prepare_features(self, df, target_col='carbon_stock'):
        """Complete feature engineering pipeline."""
        print("🔧 Starting feature engineering...")
        
        # Apply all feature engineering steps
        df_eng = self.create_spatial_features(df)
        df_eng = self.create_biomass_features(df_eng)
        df_eng = self.encode_categorical_features(df_eng)
        df_eng = self.create_interaction_features(df_eng)
        
        # Remove original categorical columns and target
        columns_to_drop = ['biomass_category']
        if 'vegetation_type' in df_eng.columns:
            columns_to_drop.append('vegetation_type')
        if 'quality_flag' in df_eng.columns:
            columns_to_drop.append('quality_flag')
        
        X = df_eng.drop(columns=[target_col] + columns_to_drop, errors='ignore')
        y = df_eng[target_col]
        
        # Remove any remaining non-numeric columns
        X = X.select_dtypes(include=[np.number])
        
        print(f"✅ Feature engineering complete. Final feature count: {X.shape[1]}")
        print(f"📊 Features: {list(X.columns)}")
        
        return X, y

# Initialize feature engineer
feature_engineer = CarbonFeatureEngineer()

# Prepare features and target
X, y = feature_engineer.prepare_features(modeling_df)

# Display feature information
print(f"\n📈 Feature matrix shape: {X.shape}")
print(f"🎯 Target vector shape: {y.shape}")
print(f"\n📋 First 5 features:")
display(X.head())

# Feature correlation with target
feature_correlations = pd.DataFrame({
    'feature': X.columns,
    'correlation_with_carbon': [X[col].corr(y) for col in X.columns]
}).sort_values('correlation_with_carbon', key=abs, ascending=False)

print("\n🔗 Top 10 features by absolute correlation with carbon stock:")
display(feature_correlations.head(10))

## 4. Train-Test Split and Data Scaling

In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, shuffle=True
)

print("📊 Dataset Split Summary:")
print(f"Training set: {X_train.shape[0]} samples")
print(f"Test set: {X_test.shape[0]} samples")
print(f"Number of features: {X_train.shape[1]}")

# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("\n✅ Feature scaling completed")
print(f"Training set mean after scaling: {X_train_scaled.mean():.4f}")
print(f"Training set std after scaling: {X_train_scaled.std():.4f}")

# Convert back to DataFrames for easier handling
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X.columns, index=X_train.index)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X.columns, index=X_test.index)

## 5. Model Training - Multiple Algorithms

In [ ]:
class CarbonModelTrainer:
    """Train and compare multiple models for carbon prediction."""
    
    def __init__(self):
        self.models = {}
        self.results = {}
        self.best_model = None
        self.best_score = -np.inf
    
    def initialize_models(self):
        """Initialize multiple regression models."""
        
        self.models = {
            'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
            'XGBoost': xgb.XGBRegressor(n_estimators=100, random_state=42, verbosity=0),
            'LightGBM': lgb.LGBMRegressor(n_estimators=100, random_state=42, verbose=-1),
            'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
            'Support Vector Machine': SVR(kernel='rbf', C=1.0),
            'Neural Network': MLPRegressor(hidden_layer_sizes=(100, 50), random_state=42, max_iter=1000),
            'Ridge Regression': Ridge(alpha=1.0),
            'Lasso Regression': Lasso(alpha=1.0)
        }
        
        print("✅ Initialized 8 different models for comparison")
    
    def train_models(self, X_train, y_train, X_test, y_test, cv_folds=5):
        """Train all models and evaluate performance."""
        
        self.results = {}
        
        for name, model in self.models.items():
            print(f"\n🚀 Training {name}...")
            
            # Train model
            model.fit(X_train, y_train)
            
            # Predictions
            y_train_pred = model.predict(X_train)
            y_test_pred = model.predict(X_test)
            
            # Calculate metrics
            train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
            test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
            train_mae = mean_absolute_error(y_train, y_train_pred)
            test_mae = mean_absolute_error(y_test, y_test_pred)
            train_r2 = r2_score(y_train, y_train_pred)
            test_r2 = r2_score(y_test, y_test_pred)
            
            # Cross-validation
            cv_scores = cross_val_score(model, X_train, y_train, cv=cv_folds, scoring='r2')
            
            # Store results
            self.results[name] = {
                'model': model,
                'train_rmse': train_rmse,
                'test_rmse': test_rmse,
                'train_mae': train_mae,
                'test_mae': test_mae,
                'train_r2': train_r2,
                'test_r2': test_r2,
                'cv_mean': cv_scores.mean(),
                'cv_std': cv_scores.std(),
                'predictions': y_test_pred
            }
            
            print(f"   ✅ {name} trained:")
            print(f"      Test R²: {test_r2:.4f}, Test RMSE: {test_rmse:.4f}")
            
            # Update best model
            if test_r2 > self.best_score:
                self.best_score = test_r2
                self.best_model = name
    
    def compare_models(self):
        """Compare model performance and create visualizations."""
        
        # Create results DataFrame
        comparison_df = pd.DataFrame({
            'Model': list(self.results.keys()),
            'Test_R2': [self.results[name]['test_r2'] for name in self.results],
            'Test_RMSE': [self.results[name]['test_rmse'] for name in self.results],
            'Test_MAE': [self.results[name]['test_mae'] for name in self.results],
            'CV_Mean_R2': [self.results[name]['cv_mean'] for name in self.results],
            'CV_Std_R2': [self.results[name]['cv_std'] for name in self.results]
        }).sort_values('Test_R2', ascending=False)
        
        print("\n" + "="*80)
        print("📊 MODEL COMPARISON RESULTS")
        print("="*80)
        display(comparison_df.round(4))
        
        print(f"\n🏆 BEST MODEL: {self.best_model} (Test R²: {self.best_score:.4f})")
        
        # Create visualizations
        self._create_comparison_plots(comparison_df)
        
        return comparison_df
    
    def _create_comparison_plots(self, comparison_df):
        """Create comparison plots for model performance."""
        
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=(
                'Model R² Scores Comparison',
                'Model RMSE Comparison',
                'Cross-Validation Performance',
                'Prediction vs Actual (Best Model)'
            )
        )
        
        # 1. R² comparison
        fig.add_trace(
            go.Bar(x=comparison_df['Model'], y=comparison_df['Test_R2'],
                  name='Test R²', marker_color='lightblue'),
            row=1, col=1
        )
        
        # 2. RMSE comparison
        fig.add_trace(
            go.Bar(x=comparison_df['Model'], y=comparison_df['Test_RMSE'],
                  name='Test RMSE', marker_color='lightcoral'),
            row=1, col=2
        )
        
        # 3. Cross-validation performance
        fig.add_trace(
            go.Scatter(x=comparison_df['Model'], y=comparison_df['CV_Mean_R2'],
                      mode='markers+lines', name='CV Mean R²',
                      error_y=dict(type='data', array=comparison_df['CV_Std_R2'], visible=True)),
            row=2, col=1
        )
        
        # 4. Prediction vs Actual for best model
        best_model_name = self.best_model
        best_predictions = self.results[best_model_name]['predictions']
        
        fig.add_trace(
            go.Scatter(x=y_test, y=best_predictions, mode='markers',
                      name=f'{best_model_name} Predictions',
                      marker=dict(color='green', opacity=0.6)),
            row=2, col=2
        )
        
        # Add perfect prediction line
        min_val = min(y_test.min(), best_predictions.min())
        max_val = max(y_test.max(), best_predictions.max())
        fig.add_trace(
            go.Scatter(x=[min_val, max_val], y=[min_val, max_val],
                      mode='lines', name='Perfect Prediction',
                      line=dict(color='red', dash='dash')),
            row=2, col=2
        )
        
        fig.update_layout(
            title_text="Model Performance Comparison",
            height=800,
            showlegend=True
        )
        
        fig.show()

# Initialize and run model trainer
model_trainer = CarbonModelTrainer()
model_trainer.initialize_models()
model_trainer.train_models(X_train_scaled_df, y_train, X_test_scaled_df, y_test)
comparison_results = model_trainer.compare_models()

## 6. Hyperparameter Optimization

In [ ]:
class HyperparameterOptimizer:
    """Optimize hyperparameters for top performing models."""
    
    def __init__(self):
        self.optimized_models = {}
        self.best_params = {}
    
    def optimize_random_forest(self, X_train, y_train, cv_folds=5):
        """Optimize Random Forest hyperparameters."""
        
        param_grid = {
            'n_estimators': [50, 100, 200],
            'max_depth': [10, 20, 30, None],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4],
            'max_features': ['sqrt', 'log2']
        }
        
        rf = RandomForestRegressor(random_state=42)
        
        # Use RandomizedSearchCV for faster optimization
        search = RandomizedSearchCV(
            rf, param_grid, n_iter=50, cv=cv_folds, 
            scoring='r2', random_state=42, n_jobs=-1
        )
        
        print("🔍 Optimizing Random Forest...")
        search.fit(X_train, y_train)
        
        self.optimized_models['Random Forest'] = search.best_estimator_
        self.best_params['Random Forest'] = search.best_params_
        
        print(f"✅ Best Random Forest R²: {search.best_score_:.4f}")
        print(f"📋 Best parameters: {search.best_params_}")
        
        return search.best_estimator_
    
    def optimize_xgboost(self, X_train, y_train, cv_folds=5):
        """Optimize XGBoost hyperparameters."""
        
        param_grid = {
            'n_estimators': [50, 100, 200],
            'max_depth': [3, 6, 9],
            'learning_rate': [0.01, 0.1, 0.2],
            'subsample': [0.8, 0.9, 1.0],
            'colsample_bytree': [0.8, 0.9, 1.0]
        }
        
        xgb_model = xgb.XGBRegressor(random_state=42, verbosity=0)
        
        search = RandomizedSearchCV(
            xgb_model, param_grid, n_iter=50, cv=cv_folds,
            scoring='r2', random_state=42, n_jobs=-1
        )
        
        print("\n🔍 Optimizing XGBoost...")
        search.fit(X_train, y_train)
        
        self.optimized_models['XGBoost'] = search.best_estimator_
        self.best_params['XGBoost'] = search.best_params_
        
        print(f"✅ Best XGBoost R²: {search.best_score_:.4f}")
        print(f"📋 Best parameters: {search.best_params_}")
        
        return search.best_estimator_
    
    def optimize_all_models(self, X_train, y_train, top_n=3):
        """Optimize top N performing models."""
        
        print("="*80)
        print("🎯 HYPERPARAMETER OPTIMIZATION")
        print("="*80)
        
        # Get top models from previous results
        top_models = []
        if hasattr(model_trainer, 'results'):
            sorted_models = sorted(
                model_trainer.results.items(), 
                key=lambda x: x[1]['test_r2'], 
                reverse=True
            )
            top_models = [name for name, _ in sorted_models[:top_n]]
        
        print(f"Optimizing top {len(top_models)} models: {top_models}")
        
        # Optimize each top model
        for model_name in top_models:
            if model_name == 'Random Forest':
                self.optimize_random_forest(X_train, y_train)
            elif model_name == 'XGBoost':
                self.optimize_xgboost(X_train, y_train)
            elif model_name == 'LightGBM':
                # Add LightGBM optimization if needed
                pass
    
    def evaluate_optimized_models(self, X_test, y_test):
        """Evaluate optimized models on test set."""
        
        print("\n" + "="*80)
        print("📊 OPTIMIZED MODELS EVALUATION")
        print("="*80)
        
        optimized_results = {}
        
        for name, model in self.optimized_models.items():
            y_pred = model.predict(X_test)
            
            test_r2 = r2_score(y_test, y_pred)
            test_rmse = np.sqrt(mean_squared_error(y_test, y_pred))
            test_mae = mean_absolute_error(y_test, y_pred)
            
            optimized_results[name] = {
                'test_r2': test_r2,
                'test_rmse': test_rmse,
                'test_mae': test_mae,
                'model': model
            }
            
            print(f"\n{name} - Optimized:")
            print(f"  Test R²: {test_r2:.4f}")
            print(f"  Test RMSE: {test_rmse:.4f}")
            print(f"  Test MAE: {test_mae:.4f}")
            
            # Compare with baseline
            if hasattr(model_trainer, 'results') and name in model_trainer.results:
                baseline_r2 = model_trainer.results[name]['test_r2']
                improvement = test_r2 - baseline_r2
                print(f"  Improvement: {improvement:+.4f}")
        
        return optimized_results

# Run hyperparameter optimization
optimizer = HyperparameterOptimizer()
optimizer.optimize_all_models(X_train_scaled_df, y_train)
optimized_results = optimizer.evaluate_optimized_models(X_test_scaled_df, y_test)

## 7. Feature Importance Analysis

In [ ]:
class FeatureImportanceAnalyzer:
    """Analyze feature importance across different models."""
    
    def __init__(self, feature_names):
        self.feature_names = feature_names
        self.importance_results = {}
    
    def calculate_importance(self, models_dict, X_test, y_test):
        """Calculate feature importance for multiple models."""
        
        print("🔍 Calculating feature importance...")
        
        for name, model_info in models_dict.items():
            model = model_info['model']
            
            # Different importance calculation methods based on model type
            if hasattr(model, 'feature_importances_'):
                # Tree-based models
                importance = model.feature_importances_
            else:
                # For linear models and others, use permutation importance
                perm_importance = permutation_importance(
                    model, X_test, y_test, n_repeats=10, random_state=42
                )
                importance = perm_importance.importances_mean
            
            # Create importance DataFrame
            importance_df = pd.DataFrame({
                'feature': self.feature_names,
                'importance': importance
            }).sort_values('importance', ascending=False)
            
            self.importance_results[name] = importance_df
            
            print(f"✅ {name} feature importance calculated")
    
    def plot_importance_comparison(self, top_n=15):
        """Create comparison plots of feature importance."""
        
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=(
                'Top Features - Random Forest',
                'Top Features - XGBoost',
                'Feature Importance Comparison',
                'Consensus Top Features'
            )
        )
        
        # Plot individual model importances
        model_positions = [('Random Forest', 1, 1), ('XGBoost', 1, 2)]
        
        for model_name, row, col in model_positions:
            if model_name in self.importance_results:
                importance_df = self.importance_results[model_name].head(top_n)
                
                fig.add_trace(
                    go.Bar(x=importance_df['importance'], 
                          y=importance_df['feature'],
                          orientation='h',
                          name=model_name),
                    row=row, col=col
                )
        
        # Feature importance comparison
        comparison_data = []
        for model_name in self.importance_results:
            importance_df = self.importance_results[model_name]
            # Normalize importance
            importance_df['importance_norm'] = importance_df['importance'] / importance_df['importance'].max()
            comparison_data.append(importance_df.set_index('feature')['importance_norm'])
        
        comparison_df = pd.concat(comparison_data, axis=1, keys=self.importance_results.keys())
        top_features = comparison_df.mean(axis=1).sort_values(ascending=False).head(top_n).index
        
        for model_name in self.importance_results:
            fig.add_trace(
                go.Scatter(x=comparison_df.loc[top_features, model_name],
                          y=top_features,
                          mode='markers',
                          name=model_name),
                row=2, col=1
            )
        
        # Consensus top features
        consensus_importance = comparison_df.mean(axis=1).sort_values(ascending=False).head(top_n)
        
        fig.add_trace(
            go.Bar(x=consensus_importance.values,
                  y=consensus_importance.index,
                  orientation='h',
                  name='Consensus',
                  marker_color='purple'),
            row=2, col=2
        )
        
        fig.update_layout(
            title_text="Feature Importance Analysis",
            height=800,
            showlegend=True
        )
        
        fig.show()
        
        # Print top features
        print("\n🏆 CONSENSUS TOP 10 FEATURES:")
        for i, (feature, importance) in enumerate(consensus_importance.head(10).items(), 1):
            print(f"{i:2d}. {feature}: {importance:.4f}")
        
        return consensus_importance

# Analyze feature importance
importance_analyzer = FeatureImportanceAnalyzer(X.columns)

# Combine original and optimized models
all_models = {}
if hasattr(model_trainer, 'results'):
    all_models.update(model_trainer.results)
if hasattr(optimizer, 'optimized_models'):
    for name, model in optimizer.optimized_models.items():
        all_models[name + ' (Optimized)'] = {'model': model}

importance_analyzer.calculate_importance(all_models, X_test_scaled_df, y_test)
consensus_features = importance_analyzer.plot_importance_comparison()

## 8. Model Evaluation and Final Selection

In [ ]:
def comprehensive_model_evaluation(best_model, X_test, y_test, feature_names):
    """Perform comprehensive evaluation of the best model."""
    
    print("="*80)
    print("🎯 COMPREHENSIVE MODEL EVALUATION")
    print("="*80)
    
    # Predictions
    y_pred = best_model.predict(X_test)
    
    # Calculate metrics
    metrics = {
        'R² Score': r2_score(y_test, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y_test, y_pred)),
        'MAE': mean_absolute_error(y_test, y_pred),
        'MAPE': np.mean(np.abs((y_test - y_pred) / y_test)) * 100
    }
    
    print("\n📊 Performance Metrics:")
    for metric, value in metrics.items():
        print(f"  {metric}: {value:.4f}")
    
    # Residual analysis
    residuals = y_test - y_pred
    
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            'Predicted vs Actual Values',
            'Residual Distribution',
            'Residuals vs Predicted',
            'Q-Q Plot of Residuals'
        )
    )
    
    # 1. Predicted vs Actual
    fig.add_trace(
        go.Scatter(x=y_test, y=y_pred, mode='markers',
                  name='Predictions', marker=dict(color='blue', opacity=0.6)),
        row=1, col=1
    )
    
    # Perfect prediction line
    min_val = min(y_test.min(), y_pred.min())
    max_val = max(y_test.max(), y_pred.max())
    fig.add_trace(
        go.Scatter(x=[min_val, max_val], y=[min_val, max_val],
                  mode='lines', name='Perfect',
                  line=dict(color='red', dash='dash')),
        row=1, col=1
    )
    
    # 2. Residual distribution
    fig.add_trace(
        go.Histogram(x=residuals, name='Residuals',
                    marker_color='lightcoral'),
        row=1, col=2
    )
    
    # 3. Residuals vs Predicted
    fig.add_trace(
        go.Scatter(x=y_pred, y=residuals, mode='markers',
                  name='Residuals', marker=dict(color='green', opacity=0.6)),
        row=2, col=1
    )
    
    # Zero residual line
    fig.add_trace(
        go.Scatter(x=[min_val, max_val], y=[0, 0],
                  mode='lines', name='Zero',
                  line=dict(color='red', dash='dash')),
        row=2, col=1
    )
    
    # 4. Q-Q plot
    from scipy import stats
    qq_data = stats.probplot(residuals, dist="norm")
    
    fig.add_trace(
        go.Scatter(x=qq_data[0][0], y=qq_data[0][1], mode='markers',
                  name='Residuals', marker=dict(color='purple')),
        row=2, col=2
    )
    
    # Q-Q line
    fig.add_trace(
        go.Scatter(x=qq_data[0][0], y=qq_data[1][0] * qq_data[0][0] + qq_data[1][1],
                  mode='lines', name='Normal',
                  line=dict(color='red', dash='dash')),
        row=2, col=2
    )
    
    fig.update_layout(
        title_text="Comprehensive Model Diagnostics",
        height=700
    )
    
    fig.show()
    
    # Statistical tests
    print("\n📈 Statistical Tests:")
    print(f"  Residual Mean: {residuals.mean():.6f}")
    print(f"  Residual Std: {residuals.std():.6f}")
    print(f"  Shapiro-Wilk Normality Test p-value: {stats.shapiro(residuals)[1]:.4f}")
    
    return metrics, residuals

# Select best model for final evaluation
best_model_name = model_trainer.best_model
if best_model_name + ' (Optimized)' in optimizer.optimized_models:
    final_model = optimizer.optimized_models[best_model_name + ' (Optimized)']
    print(f"🎯 Using optimized {best_model_name} for final evaluation")
else:
    final_model = model_trainer.results[best_model_name]['model']
    print(f"🎯 Using baseline {best_model_name} for final evaluation")

# Comprehensive evaluation
final_metrics, final_residuals = comprehensive_model_evaluation(
    final_model, X_test_scaled_df, y_test, X.columns
)

## 9. Model Saving and Deployment Preparation

In [ ]:
def save_model_pipeline(final_model, feature_engineer, scaler, X_columns, metrics):
    """Save the complete model pipeline for deployment."""
    
    import os
    import json
    
    # Create models directory
    os.makedirs('models', exist_ok=True)
    
    # Save model
    model_path = 'models/carbon_sequestration_model.pkl'
    joblib.dump(final_model, model_path)
    
    # Save scaler
    scaler_path = 'models/scaler.pkl'
    joblib.dump(scaler, scaler_path)
    
    # Save feature engineer
    feature_engineer_path = 'models/feature_engineer.pkl'
    joblib.dump(feature_engineer, feature_engineer_path)
    
    # Save feature names
    feature_names_path = 'models/feature_names.json'
    with open(feature_names_path, 'w') as f:
        json.dump(list(X_columns), f)
    
    # Save model metrics
    metrics_path = 'models/model_metrics.json'
    with open(metrics_path, 'w') as f:
        json.dump(metrics, f, indent=2)
    
    # Save model info
    model_info = {
        'model_type': type(final_model).__name__,
        'training_date': pd.Timestamp.now().isoformat(),
        'feature_count': len(X_columns),
        'performance_metrics': metrics
    }
    
    model_info_path = 'models/model_info.json'
    with open(model_info_path, 'w') as f:
        json.dump(model_info, f, indent=2)
    
    print("✅ Model pipeline saved successfully!")
    print(f"📁 Saved files in 'models/' directory:")
    print(f"   - carbon_sequestration_model.pkl (Trained model)")
    print(f"   - scaler.pkl (Feature scaler)")
    print(f"   - feature_engineer.pkl (Feature engineering pipeline)")
    print(f"   - feature_names.json (Feature names)")
    print(f"   - model_metrics.json (Performance metrics)")
    print(f"   - model_info.json (Model information)")
    
    return model_info

# Save the complete pipeline
model_info = save_model_pipeline(
    final_model, feature_engineer, scaler, X.columns, final_metrics
)

# Final summary
print("\n" + "="*80)
print("🎉 MODEL DEVELOPMENT COMPLETED SUCCESSFULLY!")
print("="*80)
print(f"\n📊 FINAL MODEL PERFORMANCE:")
print(f"   Model Type: {model_info['model_type']}")
print(f"   R² Score: {final_metrics['R² Score']:.4f}")
print(f"   RMSE: {final_metrics['RMSE']:.4f}")
print(f"   MAE: {final_metrics['MAE']:.4f}")
print(f"   Number of Features: {model_info['feature_count']}")
print(f"\n💡 The model is ready for carbon sequestration prediction!")
print("   Use the saved pipeline for making predictions on new data.")